In [4]:
import json
from collections import Counter
from pathlib import Path
import pandas as pd

# 1. 특정 폴더 경로 설정
folder_path = Path('/Users/seunghyunmoon/Code/Toy/vote_player_server/data/candidates')  # ← 여기 본인 경로로 수정하세요

# 결과 기록용 리스트
summary_records = []
empty_bills_files = []

# 2. 모든 JSON 파일 순회
json_files = list(folder_path.glob('*.json'))
print(f"총 {len(json_files)}개의 JSON 파일을 처리합니다.\n")

for file_path in json_files:
    print(f"처리 중: {file_path.name}")
    
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # 기존 컬럼 제거
    data.pop('collabillsStatusStatistics', None)
    data.pop('collabillsNthStatistics', None)

    collabills = data.get('collabills', [])
    collabills_count = len(collabills)

    if collabills_count == 0:
        print(f"  ⚠️  경고: 이 파일은 collabills 데이터가 없습니다!")
        empty_bills_files.append(file_path.name)

    # ✅ collabillsNthStatistics 집계
    nth_set = set()
    status_counter = Counter()

    for collabill in collabills:
        nth = (collabill.get('nth') or '').replace('제', '').replace('대', '')
        status = (collabill.get('status') or '').strip()

        if nth:
            nth_set.add(f"{nth}대")
        if nth and status:
            status_counter[(nth, status)] += 1

    # 결과 포맷
    data['collabillsNthStatistics'] = sorted(nth_set)
    data['collabillsStatusStatistics'] = [
        {'nth': f"{nth}대", 'name': status, 'value': count}
        for (nth, status), count in status_counter.items()
    ]

    # 저장
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    print(f"  - 저장 완료: {file_path.name}\n")

    summary_records.append({
        '파일명': file_path.name,
        'collabills 항목 수': collabills_count,
        '통계 항목 수': len(data['collabillsStatusStatistics']),
        '회기 수': len(data['collabillsNthStatistics'])
    })

# 요약 출력
summary_df = pd.DataFrame(summary_records)
summary_df = summary_df.sort_values(by='통계 항목 수', ascending=False)

print("✅ 처리 결과 요약:")
display(summary_df)

if empty_bills_files:
    print("\n⚠️ collabills 데이터가 없는 파일 목록:")
    for filename in empty_bills_files:
        print(f" - {filename}")
else:
    print("\n✅ 모든 파일에 collabills 데이터가 존재합니다!")


총 16개의 JSON 파일을 처리합니다.

처리 중: KangSeungKyoo.json
  - 저장 완료: KangSeungKyoo.json

처리 중: KANGYUJUNG.json
  - 저장 완료: KANGYUJUNG.json

처리 중: KANGKYUNGSOOK.json
  - 저장 완료: KANGKYUNGSOOK.json

처리 중: KWAKKYUTAEK.json
  - 저장 완료: KWAKKYUTAEK.json

처리 중: KANGSUNYOUNG.json
  - 저장 완료: KANGSUNYOUNG.json

처리 중: KANGMINKUK.json
  - 저장 완료: KANGMINKUK.json

처리 중: KANGSUNWOO.json
  - 저장 완료: KANGSUNWOO.json

처리 중: KANGJUNHYEON.json
  - 저장 완료: KANGJUNHYEON.json

처리 중: KANGDAESIK.json
  - 저장 완료: KANGDAESIK.json

처리 중: KANGHOONSIK.json
  - 저장 완료: KANGHOONSIK.json

처리 중: KUJAKEUN.json
  - 저장 완료: KUJAKEUN.json

처리 중: KANGMYOUNGGU.json
  - 저장 완료: KANGMYOUNGGU.json

처리 중: KWAKSANGEON.json
  - 저장 완료: KWAKSANGEON.json

처리 중: KOMINJUNG.json
  - 저장 완료: KOMINJUNG.json

처리 중: KOHDONGJIN.json
  - 저장 완료: KOHDONGJIN.json

처리 중: KANGDEUKGU.json
  - 저장 완료: KANGDEUKGU.json

✅ 처리 결과 요약:


,파일명,collabills 항목 수,통계 항목 수,회기 수
9,KANGHOONSIK.json,2379,17,3
7,KANGJUNHYEON.json,1758,11,2
8,KANGDAESIK.json,1378,11,2
10,KUJAKEUN.json,2573,11,2
0,KangSeungKyoo.json,942,10,2
5,KANGMINKUK.json,507,10,2
6,KANGSUNWOO.json,1406,10,2
15,KANGDEUKGU.json,1608,10,2
13,KOMINJUNG.json,550,9,2
1,KANGYUJUNG.json,487,5,1



✅ 모든 파일에 collabills 데이터가 존재합니다!
